**netCDF Setup Utility**

Use this notebook to set up empty netCDF files to be loaded onto each vessel. Currently, this setup utility only supports the use of two vessels.

In the cell below, please enter the required input files and runtime parameters or leave them as their default values.

In [1]:
# Survey area polygon file
survey_area_file = 'surveyAreas/mp_polygon.poly'

# Output netCDF files for each vessel
vesselA_output_file = 'output/vA_model.nc'
vesselB_output_file = 'output/vB_model.nc'

# Geodetic Parameters
input_geodetics = 'EPSG:4326'
project_geodetics = 'EPSG:32619'

# Vessel names, types, and equipment
vesselA_name = 'Kelp King'
vesselA_make = 'OSP Vessel'
vesselA_GNSS_type = 'single_code'
vesselA_sonar_type = 'singlebeam'

vesselB_name = 'Deux Banane'
vesselB_make = 'Seafloor Hydrone'
vesselB_GNSS_type = 'single_multi'
vesselB_sonar_type = 'singlebeam'

# Survey total propagated uncertainty parameters in meters
allowable_sigmaVertical = 0.15

vesselA_sigmaVertical = 0.1
vesselA_sigmaHorizontal = 0.2

vesselB_sigmaVertical = 0.5
vesselB_sigmaHorizontal = 1.5

# Safe area parameters in meters
vesselA_min_safe_depth = 2
vesselB_min_safe_depth = 1

Import libraries.

In [2]:
import geopandas
from shapely.geometry import Point, box
import pyproj
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import Polygon, Point
from shapely.ops import transform
import pandas as pd
from shapely.geometry import Polygon
import numpy as np
from shapely.prepared import prep
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'

import xarray as xr 
import csv
import math

Set up the geodetic transformation to be used to transform from latitude and longitude to northings and eastings.

In [3]:
wgs84 = pyproj.CRS(input_geodetics)
utm = pyproj.CRS(project_geodetics)

projection = pyproj.Transformer.from_crs(wgs84, utm, always_xy=True).transform

Import survey area, transform to UTM, find the home point and bounding box of the survey area.

In [4]:
filename = 'surveyAreas/mp_polygon.poly'

survey_area_poly_list = []

# Extract the points from the .poly file.
with open(filename, 'r') as csv_file:
    next(csv_file)
    reader = csv.reader(csv_file, delimiter=' ')
    for row in reader:
        survey_area_poly_list.append([float(row[1]), float(row[0])])

# Create shapely polygon from survey boundary
survey_area_poly_input = Polygon(survey_area_poly_list)

# Reproject from WGS84 to UTM
survey_area_poly_project = transform(projection, survey_area_poly_input)

# Extract the northing and easting points from the reprojected polygon and get rounded values into pandas dataframe
survey_area_easts_array, survey_area_norths_array = survey_area_poly_project.exterior.coords.xy
survey_area_easts = list(survey_area_easts_array)
survey_area_norths = list(survey_area_norths_array)
survey_area_bounds = pd.DataFrame({'x': survey_area_easts, 'y': survey_area_norths})
survey_area_bounds['x_r'] = survey_area_bounds['x'].round()
survey_area_bounds['y_r'] = survey_area_bounds['y'].round()

# Find the home point for the survey area for indexing purposes and the away point to define maximum extent and the ranges
home = (int(survey_area_bounds.x_r.min()), int(survey_area_bounds.y_r.min())) # east, north
away = (int(survey_area_bounds.x_r.max() - home[0]), int(survey_area_bounds.y_r.max() - home[1])) # east, north
east_range = range(0, away[1], 1)
north_range = range(0, away[0], 1)


Create empty numpy arrays, then put them in data arrays, and put those in a dataset. Also assign metadata values. 

Initially for each vessel there will be only one depth hypothesis layer, a soundings layer, an uncertainty layer, and an M2 layer, along with a safe/not safe layer. Additional layers will be added, starting with a second depth hypothesis layer

In [5]:
# Set up empty numpy arrays then fill with NaNs
empty_hypo1A = np.empty((away[1], away[0], 5))
empty_hypo1B = np.empty((away[1], away[0], 5))
empty_hypo1A[:] = np.nan
empty_hypo1B[:] = np.nan

# Put the empty arrays into xarray dataArrays and assign metadata values from parameters
vA_hypo1 = xr.DataArray(empty_hypo1A, 
                        dims=('north', 'east', 'layer'), 
                        coords=[east_range, north_range, ['depth', 'soundings', 'M2', 'stdev', 'safe']])
vA_hypo1.attrs['vessel_name'] = vesselA_name
vA_hypo1.attrs['vessel_make'] = vesselA_make
vA_hypo1.attrs['vessel_GNSS_type'] = vesselA_GNSS_type
vA_hypo1.attrs['vessel_sonar_type'] = vesselA_sonar_type
vA_hypo1.attrs['sigmaVertical'] = vesselA_sigmaVertical
vA_hypo1.attrs['sigmaHorizontal'] = vesselA_sigmaHorizontal
vA_hypo1.attrs['min_safe_depth'] = vesselA_min_safe_depth

vB_hypo1 = xr.DataArray(empty_hypo1B, 
                        dims=('north', 'east', 'layer'), 
                        coords=[east_range, north_range, ['depth', 'soundings', 'M2', 'stdev', 'safe']])
vB_hypo1.attrs['vessel_name'] = vesselB_name
vB_hypo1.attrs['vessel_make'] = vesselB_make
vB_hypo1.attrs['vessel_GNSS_type'] = vesselB_GNSS_type
vB_hypo1.attrs['vessel_sonar_type'] = vesselB_sonar_type
vB_hypo1.attrs['sigmaVertical'] = vesselB_sigmaVertical
vB_hypo1.attrs['sigmaHorizontal'] = vesselB_sigmaHorizontal
vB_hypo1.attrs['min_safe_depth'] = vesselB_min_safe_depth

# Create a Dataset to serve as a template, then save a copy for each vessel
vT = xr.Dataset(dict(vA_hypo1=vA_hypo1, vB_hypo1=vB_hypo1))

vT.attrs['depth_units'] = 'meters'
vT.attrs['uncertainty_units'] = 'meters'
vT.attrs['allowable_sigmaVertical'] = allowable_sigmaVertical
vT.attrs['home_point'] = home
vT.attrs['away_point'] = away

vT.to_netcdf(path=vesselA_output_file)
vT.to_netcdf(path=vesselB_output_file)

In [6]:
vT

<xarray.Dataset> Size: 3MB
Dimensions:   (north: 175, east: 247, layer: 5)
Coordinates:
  * north     (north) int32 700B 0 1 2 3 4 5 6 7 ... 168 169 170 171 172 173 174
  * east      (east) int32 988B 0 1 2 3 4 5 6 7 ... 240 241 242 243 244 245 246
  * layer     (layer) <U9 180B 'depth' 'soundings' 'M2' 'stdev' 'safe'
Data variables:
    vA_hypo1  (north, east, layer) float64 2MB nan nan nan nan ... nan nan nan
    vB_hypo1  (north, east, layer) float64 2MB nan nan nan nan ... nan nan nan
Attributes:
    depth_units:              meters
    uncertainty_units:        meters
    allowable_sigmaVertical:  0.15
    home_point:               (666675, 5073053)
    away_point:               (247, 175)